## Article scraping ##

In [224]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import os 
import requests as request

In [38]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Sangeetha\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [39]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [40]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [41]:
# div class ='content_title' is inside div class='list_text'

slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Wins 4 Webbys, 4 People's Voice Awards</div>

In [42]:
# Use the parent element to find the first a tag and save it as `news_title`
# .get_text GET TEXT AND NOT THE HTML TAGS
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA Wins 4 Webbys, 4 People's Voice Awards"

In [43]:
# Use the parent element to find the paragraph text
# 'div', class_='article_teaser_body' is inside div class ="col-md-8"
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA\'s Global Climate Change website and Solar System Interactive.'

### JPL Space Images Featured Image

In [44]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [45]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [46]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [47]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='headerimage fade-in').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [48]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Mars Facts

In [49]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [50]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [51]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

####4 pages 
base_url =  https://marshemispheres.com/
link_1 =https://marshemispheres.com/cerberus.html
link_2 =https://marshemispheres.com/schiaparelli.html
link_3 =https://marshemispheres.com/syrtis.html
link_4 =https://marshemispheres.com/valles.html

In [257]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

response = request.get(url)

In [258]:

html = browser.html
soup_obj = soup(html, 'html.parser')


In [259]:
# extract in format hemisphere_image_urls = [ hemispheres = {title:"",image:"url"}]

In [260]:
# Create a list to hold the images and titles.
hemisphere_image_urls = []

# Get a list of all of the hemispheres will go to each .html file length of this is 4 so loop go thru 4 times
links = browser.find_by_css('a.product-item img')

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # find link and click
    browser.find_by_css('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    element = browser.find_link_by_text('Sample').first
    img_url = element['href']
    title = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere["img_url"] = img_url
    hemisphere["title"] = title
    hemisphere_image_urls.append(hemisphere)
    # Finally, we navigate bac,
    browser.back()

In [261]:
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [262]:
# 5. Quit the browser
browser.quit()